<center><h1>ETL-Pipeline</h1></center>

<center><h1>Import, Clean, Aggregate</h1></center>

### Package & Environment Dependencies

In [39]:
import pandas as pd
import datetime
import sqlalchemy as db
from config import password

pd.set_option('display.max_columns', 500)

### Load Data

In [2]:
stocks = pd.read_csv('all_stocks_5yr.csv')

In [3]:
terror = pd.read_csv('globalterrorismdb_0718dist.csv', encoding = "ISO-8859-1")

C:\Users\TPCal\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
flights = pd.read_csv('flights.csv')

C:\Users\TPCal\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Terrorism Data

In [5]:
# Terror
terror_ = terror.loc[terror['imonth'] > 0].loc[terror['iday'] > 0].copy()

def create_date_terror(row):
    
    return datetime.date(row['iyear'], row['imonth'], row['iday'])

terror_['date'] = terror_.apply(lambda x: create_date_terror(x), axis=1)

terror_us = terror_[terror_['country_txt'] == 'United States']

terror_df = terror_us[[
    'provstate', 'city', 'latitude', 'longitude', 'summary', 'success', 'suicide', 
    'attacktype1_txt', 'targsubtype1_txt', 'target1', 'gname', 'motive', 'weapsubtype1_txt', 
    'weapdetail', 'propextent_txt', 'propcomment', 'date'
]]

terror_df = terror_df.rename(columns={
    'provstate': 'state',
    'attacktype1_txt': 'attack_type',
    'targsubtype1_txt': 'target_type',
    'target1': 'target',
    'gname': 'group_name',
    'weapsubtype1_txt': 'weapon_type',
    'weapdetail': 'weapon_detail',
    'propextent_txt': 'property_damage',
    'propcomment': 'property_comment'    
})

terror_df['date'] = pd.to_datetime(terror_df['date'])
terror_df = terror_df[(terror_df['date'] >= '2015-01-01') & (terror_df['date'] <= '2015-12-31')]

terror_grouped = terror_df.groupby(['date', 'state', 'city']).agg({'success': 'sum', 'suicide': 'sum'}).reset_index()

### Flight Data

In [6]:
# Flights
def create_date_flights(row):
    
    return datetime.date(row['YEAR'], row['MONTH'], row['DAY'])

flights['date'] = flights.apply(lambda x: create_date_flights(x), axis=1)
flights['date'] = pd.to_datetime(flights['date'])

flights_clean = flights[[
    'date', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'AIRLINE', 'FLIGHT_NUMBER'
]]

# Grouped by Date - 2015
flights_grouped = round(flights_clean.groupby(['date']).agg({'DEPARTURE_DELAY': 'mean',
                                                  'ARRIVAL_DELAY': 'mean'}).reset_index(), 2).rename(columns={
    'DEPARTURE_DELAY': 'departure_delay',
    'ARRIVAL_DELAY': 'arrival_delay'
})

### Stock Data

In [7]:
stocks_clean = stocks[
    (stocks['date'] >= '2015-01-01') & (stocks['date'] <= '2015-12-31')
].reset_index().drop(columns=['index'])

stocks_grouped = round(stocks_clean.groupby('date').agg({
    'open': 'mean', 
    'high': 'mean', 
    'low': 'mean', 
    'close': 'mean', 
    'volume': 'sum'
}), 2).reset_index()

stocks_grouped['date'] = pd.to_datetime(stocks_grouped['date'])

### Data Export to CSV

In [21]:
# Export to CSV
flights_grouped.to_csv('flights_grouped.csv', index=False)
terror_grouped.to_csv('terror_grouped.csv', index=False)
stocks_grouped.to_csv('stocks_grouped.csv', index=False)

### Inspect the 3 Datasets

In [22]:
flights_grouped.head()

,date,departure_delay,arrival_delay
0,2015-01-01,9.61,5.35
1,2015-01-02,12.65,9.84
2,2015-01-03,25.17,25.46
3,2015-01-04,31.57,31.98
4,2015-01-05,21.12,18.81


In [23]:
terror_grouped.head()

,date,state,city,success,suicide
0,2015-01-06,Colorado,Colorado Springs,1,0
1,2015-02-10,North Carolina,Chapel Hill,1,0
2,2015-02-17,Florida,Melbourne,1,0
3,2015-02-18,Michigan,Kalamazoo,1,0
4,2015-02-23,Ohio,Kent,1,0


In [24]:
stocks_grouped.head()

,date,open,high,low,close,volume
0,2015-01-02,82.80,83.43,81.65,82.49,1477334403
1,2015-01-05,82.06,82.53,80.57,81.05,2180602024
2,2015-01-06,81.19,81.82,79.57,80.28,2579398620
3,2015-01-07,80.92,81.92,80.19,81.34,2139799130
4,2015-01-08,81.91,83.13,81.44,82.67,2241965877


In [25]:
len(stocks_grouped), len(terror_grouped), len(flights_grouped)

(252, 36, 365)

In [26]:
flights_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 3 columns):
date               365 non-null datetime64[ns]
departure_delay    365 non-null float64
arrival_delay      365 non-null float64
dtypes: datetime64[ns](1), float64(2)
memory usage: 8.6 KB


In [27]:
stocks_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 6 columns):
date      252 non-null datetime64[ns]
open      252 non-null float64
high      252 non-null float64
low       252 non-null float64
close     252 non-null float64
volume    252 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 11.9 KB


In [28]:
terror_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
date       36 non-null datetime64[ns]
state      36 non-null object
city       36 non-null object
success    36 non-null int64
suicide    36 non-null int64
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 1.5+ KB


### Combine Data

In [42]:
combined = pd.merge(flights_grouped, stocks_grouped, on='date', how='left')
combined = pd.merge(combined, terror_grouped, on='date', how='left')
combined.to_csv('combined_ETL_data.csv')

In [43]:
combined.head(20)

,date,departure_delay,arrival_delay,open,high,low,close,volume,state,city,success,suicide
0,2015-01-01,9.61,5.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,12.65,9.84,82.80,83.43,81.65,82.49,1.477334e+09,NaN,NaN,NaN,NaN
2,2015-01-03,25.17,25.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,31.57,31.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,21.12,18.81,82.06,82.53,80.57,81.05,2.180602e+09,NaN,NaN,NaN,NaN
5,2015-01-06,22.49,21.30,81.19,81.82,79.57,80.28,2.579399e+09,Colorado,Colorado Springs,1.0,0.0
6,2015-01-07,14.52,11.96,80.92,81.92,80.19,81.34,2.139799e+09,NaN,NaN,NaN,NaN
7,2015-01-08,16.40,13.32,81.91,83.13,81.44,82.67,2.241966e+09,NaN,NaN,NaN,NaN
8,2015-01-09,15.37,12.26,82.76,83.15,81.51,82.03,1.838031e+09,NaN,NaN,NaN,NaN
9,2015-01-10,8.20,1.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<center><h1>Insert to PostgreSQL</h1></center>

### Initialize SQLAlchemy Connection

In [40]:
db_string = 'postgresql://postgres:' + password + '@localhost:5432/ETL_Project'

engine = db.create_engine(db_string)

connection = engine.connect()

### Insert Data

In [41]:
combined.to_sql('combined_data', engine)